### Using Google Images

In [1]:
# import the necessary packages
from imutils import paths
import argparse
import requests
import cv2
import os

In [ ]:
rows = open('urls.txt').read().strip().split("\n")
total = 0

In [ ]:
# loop the URLs
for url in rows:
    try:
        # try to download the image
        r = requests.get(url, timeout=60)

        # save the image to disk
        p = os.path.sep.join([os.getcwd(), "{}.jpg".format(
            str(total).zfill(8))])
        f = open(p, "wb")
        f.write(r.content)
        f.close()

        # update the counter
        print("[INFO] downloaded: {}".format(p))
        total += 1

    # handle if any exceptions are thrown during the download process
    except:
        print("[INFO] error downloading {}...skipping".format(p))

In [ ]:
# loop over the image paths we just downloaded
for imagePath in paths.list_images(os.getcwd()):
    # initialize if the image should be deleted or not
    delete = False
 
    # try to load the image
    try:
        image = cv2.imread(imagePath)
 
        # if the image is `None` then we could not properly load it
        # from disk, so delete it
        if image is None:
            delete = True
 
    # if OpenCV cannot load the image then the image is likely
    # corrupt so we should delete it
    except:
        print("Except")
        delete = True
 
    # check to see if the image should be deleted
    if delete:
        print("[INFO] deleting {}".format(imagePath))
        os.remove(imagePath)

### Using Bing Images for depressed faces

In [2]:
# set your Microsoft Cognitive Services API key along with (1) the
# maximum number of results for a given search and (2) the group size
# for results (maximum of 50 per request)
API_KEY = "..."
MAX_RESULTS = 500
GROUP_SIZE = 50
 
# set the endpoint API URL
URL = "https://api.cognitive.microsoft.com/bing/v7.0/images/search"

In [3]:
from requests import exceptions

# when attempting to download images from the web both the Python
# programming language and the requests library have a number of
# exceptions that can be thrown so let's build a list of them now
# so we can filter on them
EXCEPTIONS = set([IOError, FileNotFoundError,
    exceptions.RequestException, exceptions.HTTPError,
    exceptions.ConnectionError, exceptions.Timeout])

In [4]:
# store the search term in a convenience variable then set the
# headers and search parameters
term = "depressed facial expressions"
headers = {"Ocp-Apim-Subscription-Key" : API_KEY}
params = {"q": term, "offset": 0, "count": GROUP_SIZE}
 
# make the search
print("[INFO] searching Bing API for '{}'".format(term))
search = requests.get(URL, headers=headers, params=params)
search.raise_for_status()
 
# grab the results from the search, including the total number of
# estimated results returned by the Bing API
results = search.json()
estNumResults = min(results["totalEstimatedMatches"], MAX_RESULTS)
print("[INFO] {} total results for '{}'".format(estNumResults, term))
 
# initialize the total number of images downloaded thus far
total = 0

[INFO] searching Bing API for 'depressed facial expressions'
[INFO] 500 total results for 'depressed facial expressions'


In [6]:
# loop over the estimated number of results in `GROUP_SIZE` groups

for offset in range(0, estNumResults, GROUP_SIZE):
    # update the search parameters using the current offset, then
    # make the request to fetch the results
    print("[INFO] making request for group {}-{} of {}...".format(
        offset, offset + GROUP_SIZE, estNumResults))
    params["offset"] = offset
    search = requests.get(URL, headers=headers, params=params)
    search.raise_for_status()
    results = search.json()
    print("[INFO] saving images for group {}-{} of {}...".format(
        offset, offset + GROUP_SIZE, estNumResults))
    
    # loop over the results
    for v in results["value"]:
        # try to download the image
        try:
            # make a request to download the image
            print("[INFO] fetching: {}".format(v["contentUrl"]))
            r = requests.get(v["contentUrl"], timeout=30)

            # build the path to the output image
            ext = v["contentUrl"][v["contentUrl"].rfind("."):]
            p = os.path.sep.join([os.getcwd(), "{}{}".format(str(total).zfill(8), ext)])
 
            # write the image to disk
            f = open(p, "wb")
            f.write(r.content)
            f.close()
 
        # catch any errors that would not unable us to download the image
        except Exception as e:
            # check to see if our exception is in our list of
            # exceptions to check for
            if type(e) in EXCEPTIONS:
                print("[INFO] skipping: {}".format(v["contentUrl"]))
                continue
                
        # try to load the image from disk
        image = cv2.imread(p)

        # if the image is `None` then we could not properly load the
        # image from disk (so it should be ignored)
        if image is None:
            print("[INFO] deleting: {}".format(p))
            os.remove(p)
            continue

        # update the counter
        total += 1

[INFO] making request for group 0-50 of 500...
[INFO] saving images for group 0-50 of 500...
[INFO] fetching: https://drlillianglassbodylanguageblog.files.wordpress.com/2010/05/conan-sad.png
[INFO] deleting: C:\Users\yantongz\Desktop\Insight\dp\00000000.png
[INFO] fetching: http://etc.usf.edu/clippix/pix/depressed-facial-expression-4_medium.jpg
[INFO] fetching: http://etc.usf.edu/clippix/pix/depressed-facial-expression_medium.jpg
[INFO] fetching: http://c8.alamy.com/comp/ARPWEG/girl-woman-depressed-sad-black-white-depression-problems-black-circles-ARPWEG.jpg
[INFO] fetching: http://thumbs.dreamstime.com/z/sad-woman-grief-sorrow-closed-eyes-portrait-attractive-mature-stressed-depressed-facial-expression-33905353.jpg
[INFO] fetching: http://etc.usf.edu/clippix/pix/depressed-facial-expression-3_medium.jpg
[INFO] fetching: https://i.pinimg.com/originals/d3/18/24/d31824a9595c0bcdbd5170be084028ae.jpg
[INFO] fetching: http://thumbs.dreamstime.com/x/depressed-woman-expression-her-face-holding-

[INFO] fetching: https://c8.alamy.com/comp/MTH0BX/close-up-portrait-of-a-beautiful-young-latin-hispanic-woman-with-a-depressed-face-looking-miserable-melancholy-and-sad-human-facial-expressions-and-MTH0BX.jpg
[INFO] fetching: https://thumbs.dreamstime.com/z/crying-woman-stressed-grief-portrait-mature-sad-unhappy-emotional-facial-expression-suffering-painful-depression-tissue-46030187.jpg
[INFO] fetching: http://www.dimitri.co.uk/emotion/emotion-images/woman-sad-depressed-expression.jpg
[INFO] fetching: http://il2.picdn.net/shutterstock/videos/7857757/thumb/1.jpg
[INFO] fetching: https://thumbs.dreamstime.com/z/sad-lonely-mature-woman-grief-depression-portrait-attractive-thoughtful-worried-depressed-facial-expression-alone-30858422.jpg
[INFO] fetching: https://c8.alamy.com/comp/MTH0BJ/close-up-portrait-of-a-beautiful-young-latin-hispanic-woman-with-a-depressed-face-looking-miserable-melancholy-and-sad-human-facial-expressions-and-MTH0BJ.jpg
[INFO] fetching: https://i.pinimg.com/original

[INFO] fetching: https://thumbs.dreamstime.com/t/negative-human-face-expressions-emotions-concept-sad-depressed-woman-crying-having-depression-113554111.jpg
[INFO] fetching: https://thumbs.dreamstime.com/z/depressed-face-5979034.jpg
[INFO] fetching: https://d2v9y0dukr6mq2.cloudfront.net/video/thumbnail/2T0t-6V/emotions-face-man-depressed-faultfinding-a-faultfinding-mans-expression-medium-close-up-shot-of-the-face-on-black-background-part-of-a-series-of-depictions-of-feelings-and-emotions-with-professional-actors_nj5wqzpk__S0000.jpg
[INFO] fetching: http://fc04.deviantart.net/fs70/f/2013/267/b/0/sad_expression_by_letsbreakx-d6nqdat.png
[INFO] fetching: https://d2v9y0dukr6mq2.cloudfront.net/video/thumbnail/2T0t-6V/emotions-face-man-depressed-slumped-a-slumped-mans-expression-medium-close-up-shot-of-the-face-on-black-background-part-of-a-series-of-depictions-of-feelings-and-emotions-with-professional-actors_vjvwdktf__S0000.jpg
[INFO] fetching: https://d2v9y0dukr6mq2.cloudfront.net/video/t

[INFO] fetching: https://st2.depositphotos.com/4421371/10498/i/950/depositphotos_104983842-stock-photo-closeup-portrait-of-a-peaceful.jpg
[INFO] fetching: https://l450v.alamy.com/450v/grexx6/upset-sad-young-boy-teen-goes-to-stress-negative-mood-portrait-with-grexx6.jpg
[INFO] fetching: http://thumbs.dreamstime.com/x/woman-grief-depression-portrait-attractive-mature-suffering-stressed-headache-sad-facial-expression-blurred-30516569.jpg
[INFO] fetching: https://listverse.com/wp-content/uploads/2013/07/Surprise-e1372986876633.jpg
[INFO] deleting: C:\Users\yantongz\Desktop\Insight\dp\00000154.jpg
[INFO] fetching: https://d2v9y0dukr6mq2.cloudfront.net/video/thumbnail/2T0t-6V/emotions-face-man-depressed-slouching-a-slouching-mans-expression-medium-close-up-shot-of-the-face-on-black-background-part-of-a-series-of-depictions-of-feelings-and-emotions-with-professional-actors_ekku4jft__F0000.png
[INFO] fetching: https://thumbs.dreamstime.com/t/young-attractive-businessman-worried-tired-stressed-

[INFO] fetching: https://thumb7.shutterstock.com/display_pic_with_logo/696460/216973135/stock-photo-depressed-gloomy-closeup-portrait-unhappy-middle-age-woman-head-on-hand-bothered-by-mistake-216973135.jpg
[INFO] fetching: https://thumb7.shutterstock.com/display_pic_with_logo/3407144/745879375/stock-photo-young-handsome-man-in-the-shirt-with-sad-face-expression-feeling-depressed-and-miserable-while-he-745879375.jpg
[INFO] fetching: https://upload.wikimedia.org/wikipedia/commons/6/65/A_woman_diagnosed_as_suffering_from_melancholia._Colour_lith_Wellcome_L0026686.jpg
[INFO] fetching: http://thumbs.dreamstime.com/z/pencil-hand-drawn-portrait-young-woman-sad-depressed-face-expression-30217600.jpg
[INFO] fetching: http://thumb9.shutterstock.com/photos/display_pic_with_logo/696460/164502236.jpg
[INFO] fetching: http://thumb7.shutterstock.com/display_pic_with_logo/696460/217501831/stock-photo-sad-thinking-closeup-portrait-headshot-depressed-alone-tired-child-resting-head-on-fist-217501831.jpg


[INFO] fetching: https://thumbs.dreamstime.com/b/teenager-girl-stress-pain-suffering-depression-sad-scared-fear-face-expression-lonely-young-looking-isolated-69487782.jpg
[INFO] fetching: http://thumb1.shutterstock.com/display_pic_with_logo/696460/172980500/stock-photo-closeup-portrait-of-dull-upset-sad-bothered-young-woman-resting-face-on-hand-really-depressed-172980500.jpg
[INFO] fetching: https://www.colourbox.com/preview/13187976-closeup-portrait-of-young-upset-sad-depressed-worried-troubled-brunette-woman-holding-her-head-with-both-hands-isolated-human-emotions-face-expressions-feelings-perception.jpg
[INFO] fetching: https://static01.nyt.com/images/2014/04/29/science/29FACE/29FACE-superJumbo.jpg
[INFO] fetching: https://thumb7.shutterstock.com/display_pic_with_logo/1390159/570357439/stock-photo-drunk-woman-alone-in-wasted-and-depressed-face-expression-holding-and-looking-thoughtful-to-scotch-570357439.jpg
[INFO] fetching: https://thumbs.dreamstime.com/z/sad-heartbroken-woman-cryi

[INFO] fetching: https://thumbs.dreamstime.com/z/head-portrait-senior-mature-old-man-his-s-looking-sad-worried-suffering-pain-depression-sadness-face-expression-104373306.jpg
[INFO] fetching: https://thumbs.dreamstime.com/z/sad-man-closeup-side-view-profile-head-shot-portrait-bothered-stressed-young-depressed-something-hands-face-isolated-40114487.jpg
[INFO] fetching: https://c8.alamy.com/comp/PW4HRC/portrait-of-senior-mature-old-businessman-looking-overwhelmed-and-depressed-suffering-at-work-in-sad-face-expression-overworked-overtime-and-aging-con-PW4HRC.jpg
[INFO] fetching: https://as2.ftcdn.net/jpg/01/92/96/69/500_F_192966990_n58G3IORxxWuy9S7sTjk7nI0jZTuJSRx.jpg
[INFO] fetching: https://ak8.picdn.net/shutterstock/videos/16553818/thumb/1.jpg
[INFO] fetching: http://1.bp.blogspot.com/-wPtH34Ql7q8/UUTQtxwHROI/AAAAAAAAANk/kcwMfyPlibM/s320/Facial+expression+depression.png
[INFO] fetching: https://thumbs.dreamstime.com/z/teenager-girl-woman-stress-pain-suffering-depression-looking-sad-lon

[INFO] fetching: http://ak2.picdn.net/shutterstock/videos/10878812/thumb/4.jpg
[INFO] fetching: https://thumbs.dreamstime.com/z/teenager-girl-stress-pain-suffering-depression-sad-scared-fear-face-expression-lonely-young-looking-isolated-69487782.jpg
[INFO] fetching: https://d2v9y0dukr6mq2.cloudfront.net/video/thumbnail/r6uQGb9/real-cuban-people-and-emotions-portrait-of-sad-young-latino-man-from-havana-cuba-looking-at-camera-with-worried-face-and-depressed-expression_r0zl0rr1g_thumbnail-small08.jpg
[INFO] fetching: http://thumb7.shutterstock.com/display_pic_with_logo/696460/178318745/stock-photo-closeup-portrait-of-dull-upset-sad-bothered-young-woman-making-puppy-dog-face-really-depressed-178318745.jpg
[INFO] fetching: http://thumb1.shutterstock.com/display_pic_with_logo/696460/182745080/stock-photo-closeup-profile-portrait-of-sad-bothered-stressed-serious-young-man-hand-on-face-depressed-about-182745080.jpg
[INFO] fetching: https://c8.alamy.com/comp/R17R29/portrait-of-older-adult-senio

[INFO] fetching: https://thumb7.shutterstock.com/display_pic_with_logo/1390159/581253145/stock-photo-young-devastated-depressed-woman-crying-sad-feeling-hurt-suffering-depression-in-sadness-emotion-581253145.jpg
[INFO] fetching: https://thumbs.dreamstime.com/b/head-portrait-senior-mature-old-man-his-s-looking-sad-worried-suffering-pain-depression-sadness-face-expression-104372986.jpg
[INFO] fetching: https://thumb9.shutterstock.com/display_pic_with_logo/163619758/631297166/stock-photo-lonely-senior-woman-portrait-sad-depressed-emotion-feelings-thoughtful-senior-old-woman-wait-631297166.jpg
[INFO] fetching: https://thumbs.dreamstime.com/z/upset-woman-closeup-portrait-dull-sad-bothered-young-resting-face-fist-depressed-something-isolated-white-37216685.jpg
[INFO] fetching: https://dorar.at/imup2/2014-10/1413715033983.jpg
[INFO] fetching: http://image.shutterstock.com/display_pic_with_logo/61118/117624346/stock-photo-depressed-and-lonely-teenage-girl-sad-expression-face-117624346.jpg
[INF

[INFO] fetching: https://image.shutterstock.com/display_pic_with_logo/696460/184569137/stock-photo-closeup-portrait-very-sad-depressed-stressed-alone-disappointed-gloomy-young-man-resting-his-184569137.jpg
[INFO] fetching: https://si.wsj.net/public/resources/images/BN-IR109_0602DE_P_20150529162320.jpg
[INFO] fetching: https://d2v9y0dukr6mq2.cloudfront.net/video/thumbnail/2T0t-6V/emotions-face-man-depressed-terrible-a-terrible-mans-expression-medium-close-up-shot-of-the-face-on-black-background-part-of-a-series-of-depictions-of-feelings-and-emotions-with-professional-actors_vjhtjeu2__S0000.jpg
[INFO] fetching: https://d2v9y0dukr6mq2.cloudfront.net/video/thumbnail/r6uQGb9/60-hispanic-people-portrait-young-sad-man-face-expression_v1ugv2wu__F0000.png
[INFO] fetching: https://get.pxhere.com/photo/person-people-girl-woman-hair-male-sadness-portrait-child-facial-expression-childhood-smile-mouth-close-up-sad-face-nose-fear-infant-toddler-eye-head-skin-expression-organ-sorrow-laughter-emotion-u